Team Balancer

In [7]:
import pandas as pd
import math
import random
from collections import defaultdict

# Function to create teams with balanced ratios
def create_balanced_teams(data):
    # Ratio requirements
    RATIO = {'ATT': 2, 'MID': 3, 'DEF': 2}
    TOTAL_RATIO = sum(RATIO.values())
    MAX_TEAM_SIZE = 7

    # Shuffle the dataset rows to randomize team assignment
    data = data.sample(frac=1, random_state=None).reset_index(drop=True)

    # Count total players by position
    position_counts = data['Position'].value_counts().to_dict()

    # Ensure there are enough players for the ratio
    total_players = len(data)
    min_teams = max(1, math.ceil(total_players / MAX_TEAM_SIZE))
    max_possible_teams = min(position_counts[pos] // RATIO[pos] for pos in RATIO)
    num_teams = min(min_teams, max_possible_teams)

    team_size = total_players // num_teams  # Approx. size per team
    extra_players = total_players % num_teams

    # Prepare to distribute players with randomness
    players_by_position = {
        pos: data[data['Position'] == pos]['Name'].tolist() for pos in RATIO
    }

    teams = defaultdict(list)
    for team_idx in range(1, num_teams + 1):
        # Fill the team based on the ratio
        for pos, count in RATIO.items():
            for _ in range(math.floor(count / TOTAL_RATIO * team_size)):
                if players_by_position[pos]:
                    teams[team_idx].append((players_by_position[pos].pop(), pos))

    # Distribute remaining players equally among teams
    remaining_players = []
    for pos, players in players_by_position.items():
        remaining_players.extend([(player, pos) for player in players])

    for idx, player in enumerate(remaining_players):
        team_idx = (idx % num_teams) + 1
        teams[team_idx].append(player)

    return teams

# Main function to load data and create teams
def main(file_path):
    try:
        data = pd.read_excel(file_path)
        if 'Name' not in data.columns or 'Position' not in data.columns:
            raise ValueError("Spreadsheet must contain 'Name' and 'Position' columns.")
    except Exception as e:
        print(f"Error loading file: {e}")
        return

    # Create balanced teams
    teams = create_balanced_teams(data)

    # Save teams to a new Excel file
    output_file = "teams_output.xlsx"
    with pd.ExcelWriter(output_file) as writer:
        for team_idx, members in teams.items():
            df = pd.DataFrame(members, columns=['Name', 'Position'])
            df.to_excel(writer, sheet_name=f'Team_{team_idx}', index=False)

    print(f"Teams have been successfully created and saved to {output_file}")

# Run the script in the notebook
file_path = "TeamSheet.xlsx"  # Replace with your spreadsheet path
main(file_path)


Teams have been successfully created and saved to teams_output.xlsx
